In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import pandas as pd

import numpy as np

In [7]:
data_dir = '/data/astro/scratch/lcabayol/Flux_estimation/img_NB685'
img_fname = 'red_NightlyR11.paucam.13564.0959.0149.FT_NB615_NB685.2470382.std.01.fits'

img_path = os.path.join(data_dir, img_fname)


In [8]:
from astropy.io import fits
from sqlalchemy.engine import create_engine
from sqlalchemy import *
from sqlalchemy.orm import *

engine = create_engine('postgresql://readonly@db03.pau.pic.es/dm')


In [9]:
def DB_query(band,fn):

    query = ("""SELECT fa.ref_id, cm.ra, cm.dec, i.filename, m.exp_num,fa.image_id, cm.r50, cm.sersic_n_gim2d, fa.flux, fa.flux_error, fa.aperture_a, fa.aperture_b, fa.aperture_theta, i.psf_fwhm, i.filter, fa.aperture_x, fa.aperture_y, cm."I_auto" """+
        """FROM mosaic as m """ +
        """JOIN image as i ON i.mosaic_id = m.id """+
        """JOIN forced_aperture as fa ON fa.image_id = i.id """+
        """JOIN COSMOS as cm ON cm.paudm_id = fa.ref_id """ +
        """WHERE fa.production_id = 948 """+
        """AND cm."I_auto" < 23 """+
        """AND cm.r50 IS NOT NULL """+
        """AND cm.zspec !=0 """+
        """AND i.filter = '%s' """%band+
        """AND sersic_n_gim2d > 0 """+
        """AND r50 > 0 """+
        """AND i.filename = '%s'"""%fn)
    return query


In [10]:
hdul = fits.open(img_path)
img = hdul[0].data
band = 'NB685'
query = DB_query(band,img_fname)

with engine.begin() as conn:
    conn.execute("SET TRANSACTION ISOLATION LEVEL SERIALIZABLE READ ONLY DEFERRABLE")
    sources = pd.read_sql(query, conn)


In [12]:
sources = sources.loc[:10].copy()

In [13]:
sources['interv'] = np.zeros(shape = len(sources))
sources.interv.where(sources.exp_num < 7582, 1, inplace=True)
sources = sources.rename(columns = {'filter':'band'})
sources['interv'] = sources.interv.astype(np.int)

/tmp/ipykernel_62/80050286.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sources['interv'] = sources.interv.astype(np.int)


In [14]:
coords_pix = pd.DataFrame(np.c_[sources.aperture_y, sources.aperture_x], columns = ['x','y'])

In [15]:
met = sources[['ref_id','image_id','r50','sersic_n_gim2d','psf_fwhm','aperture_x','aperture_y','aperture_a','aperture_b','I_auto','band','aperture_theta','interv']]

In [18]:
import lumos
model_path = '/data/astro/scratch/lcabayol/Flux_estimation/models/lumos_after/Lumos_withprof_44.pt'
net = lumos.Lumos_net(model_path)



#background_img(self, img, coords_pix, metadata)

In [19]:
df_pred = net._photometry_img( img, coords_pix, met) 

NB685_1


/nfs/pic.es/user/l/lcabayol/lumos/lumos/lumos.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata['band_int'] = self._internal_naming(metadata.band.values[0], interv) #*np.ones(shape = len(metadata))


In [20]:
df_pred['ref_id'] = met.ref_id
df_pred['image_id'] = met.image_id

In [21]:
df_pred['band'] = met.band

In [35]:
alpha_keys = ['a1','a2','a3','a4','a5']
flux_keys = ['f1','f2','f3','f4','f5']
sigma_keys = ['e1','e2','e3','e4','e5']

In [43]:
import lumos.lumos_fluxmeasurements as lumos_fluxmeasurements

In [44]:
df_pred_clear, alphas, fluxes, sigmas = lumos_fluxmeasurements.clear_catalogue(cat = df_pred, alpha_keys = alpha_keys, flux_keys=flux_keys , sigma_keys= sigma_keys)

In [45]:
cat_single_observations = lumos_fluxmeasurements.single_exposures_flux(alphas, fluxes, sigmas)

In [47]:
query = f"""
    SELECT izp.image_id, izp.zp,izp.zp_error, izp.calib_method
    FROM image_zp as izp
    WHERE izp.phot_method_id = 2
    AND izp.calib_method = 'MBE2.1_xsl'
    """
#    AND fa.flag = 0

with engine.begin() as conn:
    conn.execute("SET TRANSACTION ISOLATION LEVEL SERIALIZABLE READ ONLY DEFERRABLE")
    izp = pd.read_sql(query, conn)


In [48]:
df_pred_clear_calib = df_pred_clear.merge(izp, on = 'image_id')

In [58]:
import lumos.lumos_fluxmeasurements as lumos_fluxmeasurements

In [59]:
cat_coadds = lumos_fluxmeasurements.coadds_flux_measurements(df_pred_clear_calib,alpha_keys,flux_keys,sigma_keys)

/nfs/pic.es/user/l/lcabayol/lumos/lumos/lumos_fluxmeasurements.py:105: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  cat_coadds = pd.concat((cat_coadds,sub),0)


In [60]:
cat_coadds

,ref_id,band,flux,flux_error
0,40366.0,NB685,18.4,8.18
0,40548.0,NB685,45.0,12.8
0,40253.0,NB685,57.0,14.8
0,40710.0,NB685,51.5,14.95
0,40874.0,NB685,9.17,3.995
0,40711.0,NB685,560.9,32.65
0,39756.0,NB685,46.0,15.25
0,39922.0,NB685,69.6,14.4
0,56754.0,NB685,29.9,14.815
0,40001.0,NB685,24.7,12.68
